## Tobigs 5주차 Crawling 과제
- Selenium을 활용하여 학술연구정보서비스 RISS에서 주어진 키워드에 대한 **학위 논문**을 검색 후 **첫번째 페이지**에 존재하는 논문의 **URL**을 먼저 수집합니다. 
- 수집한 URL을 기반으로 접근하여 논문 정보 **1) 저자 2) 발행년도 3) 제목 4) 부가정보의 첫번째 항목**을 수집해주세요! 
- 주석을 읽고 주어진 코드 중간에 있는 ???와 공란을 채워주시면 됩니다!

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd

In [3]:
keyword='코로나'  #검색용 키워드
paper=[]  #결과 저장용 리스트

#### 1. 드라이버 오픈 및 Riss로 이동

In [80]:
driver = webdriver.Chrome('./chromedriver')  #드라이버 오픈
driver.get('http://www.riss.kr/index.do')   #riss로 이동

#### 2. 검색창에 접근하여 검색용 키워드 입력 및 검색

In [9]:
driver.find_element_by_id('query').send_keys(keyword)  # 검색창 접근 및 키워드 입력
driver.find_element_by_id('query').submit()   # 검색창에 검색용 키워드 검색

#### 3. 검색 화면 창에서 학위 논문 창으로 이동
![image](https://user-images.githubusercontent.com/69336270/129825390-d270bcca-003c-46f4-8bfd-14e4f1ed3f87.png)
  
    방법 1. 상단의 학위 논문 탭 클릭  
    방법 2. 검색 화면 창에서 학위논문 결과 옆 더보기 버튼 클릭

In [10]:
driver.find_element_by_xpath('//*[@id="tabMenu"]/div/ul/li[3]/a').send_keys(Keys.ENTER) 

#### 4. 주어진 키워드로 검색했을 때 학위 논문의 전체 수 구하기

![image](https://user-images.githubusercontent.com/69336270/129825100-89dd9681-4dd7-451a-804f-e0fbe437aeba.png)

In [15]:
total=driver.find_element_by_xpath('//*[@id="divContent"]/div[2]/div/div[1]/dl/dd/span/span').text  #전체 논문수
print(total)

import re
_total = re.sub(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', total)
total = int(_total)

print(total)

2,367
2367


#### 5. 첫페이지에 존재하는 논문의 상세정보 url 수집

In [21]:
html = driver.page_source
BeautifulSoup(html, 'html.parser')

page = '//*[@id="divContent"]/div[2]/div/div[3]/div[2]/ul/li[1]/div[2]/p[1]/a'   
url = driver.find_element_by_xpath(page).get_attribute('href')

```find_element_by_xpath('...')```는 parameter로 받은 xpath에 존재하는 웹페이지 Element를<br>
```selenium.webdriver.remote.webelement.WebElement```라는 객체에 담아서 반환하는 것 같다.<br>
해당 객체의 속성으로 ```id```, ```text```, ```tag_name```에는 바로 접근이 가능하다.<br>
하지만 그외 다른 속성은 ```get_attribute()```라는 메소드를 사용해야 접근 가능하다.


In [31]:
print(driver.find_element_by_xpath(page).text)
print('return value of find_element_by_xpath: ',driver.find_element_by_xpath(page))
print('return type of find_element_by_xpath: ', type(driver.find_element_by_xpath(page)))
print(driver.find_element_by_xpath(page).get_attribute('text'))
print(driver.find_element_by_xpath(page).get_attribute('href'))

코로나19 정보 이용이 예방행동의도에 미치는 영향 : 한국과 중국 대학생의 차이를 중심으로
return value of find_element_by_xpath:  <selenium.webdriver.remote.webelement.WebElement (session="407283808573eb955921a781f7fa8f44", element="05624e47-2087-481e-9689-bbc220067c05")>
return type of find_element_by_xpath:  <class 'selenium.webdriver.remote.webelement.WebElement'>
코로나19 정보 이용이 예방행동의도에 미치는 영향 : 한국과 중국 대학생의 차이를 중심으로
http://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=60c040736dc4a7ebffe0bdc3ef48d419


In [32]:
urls = []

html = driver.page_source # 페이지 소스 저장
BeautifulSoup(html, 'html.parser')

if total < 10:
    # 주어진 키워드로 검색 시 전체 논문 수가 10개 미만인 경우 - 논문 첫 페이지의 논문 수 = 전체 논문 수
    for i in range(1, total+1):  
        page = '/html/body/div[1]/div[2]/div[4]/div[2]/div/div[3]/div[2]/ul/li['+str(i)+']/div[2]/p[1]/a'
        url=driver.find_element_by_xpath(page).get_attribute('href')
        urls.append(url)
        print(url)
else:
    # 주어진 키워드로 검색 시 전체 논문 수가 10개 이상인 경우 - 논문 첫 페이지의 논문 수 = 10개
    for i in range(1, 11):
        page = '/html/body/div[1]/div[2]/div[4]/div[2]/div/div[3]/div[2]/ul/li['+str(i)+']/div[2]/p[1]/a'
        url=driver.find_element_by_xpath(page).get_attribute('href')
        urls.append(url)
        print(url)

http://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=60c040736dc4a7ebffe0bdc3ef48d419
http://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=5fc876f4af70a3a7ffe0bdc3ef48d419
http://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=bd880fa47b789373ffe0bdc3ef48d419
http://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=37bf971be792bed3ffe0bdc3ef48d419
http://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=837163f702828b8dffe0bdc3ef48d419
http://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=b90415ff8378fac2ffe0bdc3ef48d419
http://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=bc236b503a9c6dfaffe0bdc3ef48d419
http://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=9f4d0af84f537701ffe0bdc3ef48d419
http://www.riss.kr/search/detail/DetailView.do?p

#### 6-1. 저자 정보 구하기 예제
수집한 url 중 첫번째 링크의 저자 정보를 구해보자!

In [64]:
data = []  # 수집한 정보 저장용 리스트
driver.get(urls[0])   # 첫번째 링크로 이동

In [65]:
author = driver.find_element_by_xpath('//*[@id="thesisInfoDiv"]/div[2]/div[1]/ul/li[1]/div/p/a').text
data.append(author)
print(author)

도효쌍


#### 6-2. 연도 정보 구하기 예제
수집한 url 중 첫번째 링크의 연도 정보를 구해보자!

In [66]:
year = driver.find_element_by_xpath('//*[@id="thesisInfoDiv"]/div[2]/div[1]/ul/li[4]/div/p').text
data.append(year)
print(year)

2021


#### 6-3. 제목 정보 구하기 예제
수집한 url 중 첫번째 링크의 제목 정보를 구해보자!

In [67]:
title = driver.find_element_by_xpath('//*[@id="thesisInfoDiv"]/div[1]/h3').text
data.append(title)
print(title)

코로나19 정보 이용이 예방행동의도에 미치는 영향 : 한국과 중국 대학생의 차이를 중심으로 = The impact of COVID-19 news usage on the media users’ behavior of health protection : focusing on difference between Korean and Chinese college students


#### 6-4. 부가정보 정보 구하기 예제
수집한 url 중 첫번째 링크의 부가정보 정보를 구해보자!

![image](https://user-images.githubusercontent.com/69336270/129768125-b13b9554-7dba-44e0-aa27-07b901a7f50c.png)
  
부가정보에서 데이터를 수집하기 위해서는 우선 더보기 버튼을 클릭해주어야 한다. 이때, 더보기 버튼의 **클래스명**을 통해 클릭해주자!

In [68]:
# 부가정보 가져오기 - 더보기 버튼 클릭

# class를 칭할 때는 클래스 이름 앞에 .을 찍고, id를 칭할 때는 아이디 이름 앞에 #을 찍었다.
# 하지만
moreViews = driver.find_elements_by_class_name('moreView')

for moreView in moreViews:
    moreView.send_keys(Keys.ENTER)
    sleep(0.1)

In [69]:
# 부가정보 가져오기 - 첫번째 항목만 가져오기

try:
    first_info = driver.find_element_by_xpath('//*[@id="additionalInfoDiv"]/div/div[1]/div[3]/p').text
    data.append(first_info)
    print(first_info)
except:
    data.append("부가정보가 없습니다.")

통신기술의 급속한 발전과 사람들의 생활수준의 향상에 따라, 점점 더 많은 사람들이 건강과 관련된 화제에 대한 관심도 커지고 있었다. 그들이 미디어를 통해 질병 관련 지식을 얻는 것을 더 선호하며, 미디어도 시기적으로 사람들에게 정보를 제공할 수 있고 시간이나 경제 등 자원도 절약할 수 있기 때문이다. 커뮤니케이션 영역에서도 끊임없이 미디어와 질병의 상관관계를 연구하기 시작하였다. 일부 학자들은 미디어가 질병에 대한 보도 내용의 프레임과 건강 인식의 관계를 탐구하였으며, 어떤 학자들은 질병 관련 보도의 사용량과 개인의 건강보호행동의도의 관계를 파악하였다. 특히, 코로나19처럼 심각한 전염병에 관한 연구도 이루 헤아릴 수 없다. 전연병일 발생했을 때 사람들이 정보에 대한 욕구가 더욱 커졌다. 해당 위기에 대한 충분한 인식을 필요하고, 심각성, 피해 발생 가능성에 대한 판단을 내려야 한다. 또한 어떤 방법을 통해 자신의 안전을 효과적으로 지킬 수 있는지를 가장 중요한 지식이며, 사람들이 예방조치를 취하는 데 영향을 마치는 요인이 중요한 역할을 발휘한다. 선행연구를 의하면, 사람들이 질병에 대한 건강보호의지는 하당 질병 관련 정보 이용량이 영향을 미치는 것으로 발견되었다. 측 질병 관련 정보 이용량이 많을수록 건강증진의지가 더 높아진다는 결과를 도출하며, 본 연구는 보호동기이론에 근거하여 선행연구를 바탕으로 코로나19 관련 뉴스 이용정도가 이들의 예방행위의도에 미치는 영향을 검증하였다. 추가적으로 문화적 차이가 영향을 미칠 수 있는지를 검증하기 위해 한국 대학생 338명과 중국 대학생 345명 총 683명인 대학생을 대상으로 설문조사를 실시하였다. 코로나19 관련 뉴스의 이용정도를 독립변수로, 보호동기인 인지된 심각성, 인지된 취약성, 반응효능감, 자기효능감 이 네 가지 요소를 매개변수로, 예방행동의도를 종속수로 설정하였으며, 각 요소 간의 관계를 탐구하는 목적으로 가설과 문제를 설정하고 검증하였다. 최종적으로 예방행동의도에 어떤 요인이 역할을 발휘할지를 알게 된다.

만약 더보기를 펼치지 않고 ```text```를 가져 오면 어떨까 하여 실험을 해본 결과,<br>
아래와 같이 더보기가 펼쳐지기 이전의 짧은 ```text```를 반환한다.

In [70]:
driver.find_element_by_xpath('//*[@id="additionalInfoDiv"]/div/div[1]/div[2]/p').text

''

#### 6-5. 수집한 정보를 바탕으로 DataFrame 만들기

In [71]:
pd.DataFrame(data = {'저자':data[0], '발행년도':data[1], '제목':data[2], '부가정보':data[3], 'Url':urls[0]}, index = [0])

,저자,발행년도,제목,부가정보,Url
0,도효쌍,2021,코로나19 정보 이용이 예방행동의도에 미치는 영향 : 한국과 중국 대학생의 차이를 ...,"통신기술의 급속한 발전과 사람들의 생활수준의 향상에 따라, 점점 더 많은 사람들이 ...",http://www.riss.kr/search/detail/DetailView.do...


#### 7. 논문 정보 수집 과정 함수화 하기
앞선 예제를 기반으로 함수 속 ???와 공란을 채워주세요

In [77]:
def store(url):
    data=[]  #빈 리스트
    authur=""  #저자
    year=""  #연도
    title=""  #제목
    first_info=""  #부가정보
        
    #저자
    author = driver.find_element_by_xpath('//*[@id="thesisInfoDiv"]/div[2]/div[1]/ul/li[1]/div/p/a').text
        
    #연도
    year = driver.find_element_by_xpath('//*[@id="thesisInfoDiv"]/div[2]/div[1]/ul/li[4]/div/p').text
        
    #제목
    title = driver.find_element_by_xpath('//*[@id="thesisInfoDiv"]/div[1]/h3').text
             
    #부가정보    
    moreViews = driver.find_elements_by_class_name('moreView')

    for moreView in moreViews:
        moreView.send_keys(Keys.ENTER)
        sleep(0.1)

    try:
        first_info = driver.find_element_by_xpath('//*[@id="additionalInfoDiv"]/div/div[1]/div[3]/p').text
    except:
        data.append("부가정보가 없습니다.")
        
    data.append(author)
    data.append(int(year))
    data.append(title)
    data.append(first_info)
    data.append(url)
    paper.append(data)

#### 8. 수집 및 저장
해당 코드는 채점을 위한 코드로 수정하시면 안됩니다!

In [81]:
paper=[]
for url in urls:
    driver.get(url)
    store(url)
    sleep(1)

driver.close()  # 드라이버 닫기

paper_info = pd.DataFrame(paper, columns=['저자', '발행년도', '제목', '부가정보', 'Url'])
paper_info.to_csv('./paper_info.csv', encoding='utf-8')
paper_info

,저자,발행년도,제목,부가정보,Url
0,도효쌍,2021,코로나19 정보 이용이 예방행동의도에 미치는 영향 : 한국과 중국 대학생의 차이를 ...,"통신기술의 급속한 발전과 사람들의 생활수준의 향상에 따라, 점점 더 많은 사람들이 ...",http://www.riss.kr/search/detail/DetailView.do...
1,원금숙,2021,코로나19이후 서울·경기지역 영양(교)사의 직무만족도와 급식관리의 변화,"본 연구는 서울·경기지역에 근무하는 현직 학교영양(교)사 201명(서울 101명, ...",http://www.riss.kr/search/detail/DetailView.do...
2,서리라,2021,코로나19 확산 이후 전염병에 관한 질병지각이 관광 행동의도에 미치는 영향 : 보호...,현재 코로나19로 인한 팬데믹 상황 속에서 전 세계의 관광산업은 유례없는 하락세를 ...,http://www.riss.kr/search/detail/DetailView.do...
3,문다정,2021,코로나-19 바이러스 감염증(COVID-19) 치료병원 간호사의 외상 후 스트레스와...,Objective: Coronavirus infection-19 is a world...,http://www.riss.kr/search/detail/DetailView.do...
4,임동준,2021,코로나19 사태가 한국 식품시장에 미친 영향 : 식품 유통업계 구매·판매 동향을 중...,"본 연구는 코로나19 사태가 일어난 2020년의 식품시장 변화를 폭넓게 수집, 분석...",http://www.riss.kr/search/detail/DetailView.do...
5,이진희,2021,코로나19(COVID-19) 이후 도시공원 특성에 따른 상권 매출 차이에 관한 연구...,"As a result of the COVID-19 Pandemic, a major ...",http://www.riss.kr/search/detail/DetailView.do...
6,최재연,2020,코로나-19(COVID-19)로 인한 사회적 위험이 소비심리와 HMR 구매패턴에 미...,"Currently, the world is facing disruption of s...",http://www.riss.kr/search/detail/DetailView.do...
7,김윤서,2021,코로나19 이후 소비.미디어 이용행태 변화와 코로나 우울에 관한 연구 = Chang...,"본 연구결과를 요약하면 다음과 같다.\n첫째, 코로나19이후 소비행태에는 변화가 존...",http://www.riss.kr/search/detail/DetailView.do...
8,"Zhang, Yuqing",2021,코로나19 예방행위 지속의도에 영향을 미치는 요인 연구 : 건강신념모델(HBM)의 적용,사람들이 질병 혹은 위험의 특성을 지각하는 것에 따라 우리는 그들의 예방 행동을 예...,http://www.riss.kr/search/detail/DetailView.do...
9,남정희,2021,코로나19로 인한 사회적 위험 인식에 따른 피부관리실 이용실태 = Status of...,Since the World Health Organization declared a...,http://www.riss.kr/search/detail/DetailView.do...


# Next Step.
검색을 하여 ```total```이 10 이하이면 하나의 페이지만 나온다.<br>
하지만 일반적인 경우 검색을 하면 여러 페이지가 나올 것이다.<br>
검색하여 나온 논문이 10개 이상이면 여러 페이지가 나올 것이고, <br>
만약 100개 이상의 논문이 나오면 10개의 페이지가 보이고 double arrow가 나올 것이다. <br>
이를 일반화하여 더 많은 논문에 대해 크롤링 해보자.<br>

In [2]:
driver = webdriver.Chrome('./chromedriver')  #드라이버 오픈
driver.get('http://www.riss.kr/index.do')   #riss로 이동

또 다른 페이지로 이동하기 위한 페이지 번호 href의 class는 ```num```이다.<br>
하지만 정작 ```num```이라는 class name으로 elements를 다 가져 오면 페이지 내에 41개가 존대한다.<br>
41개 전부가 페이지 이동을 위한 ```href```는 아닐 것이고, 중복이 있을 수도 있다.<br>
이를 해결하기 위해 특정 조건만 만족하는 ```num```을 가져 와야 할 것이다.<br>
가령 위에서 논문의 링크를 가져온 것과 같이 경로를 이용한다던가,<br>
또는 특정 조건을 만족하는 ```num```만 ```find_elements```한다던가.<br>
아무래도 특정 조건을 만족하는 것들만 가져 오는 것이 일반화 성능에서 더 좋을 것이다.<br>

In [3]:
_nums = driver.find_elements_by_class_name('num')
len(_nums)

41

In [5]:
for num in _nums:
    print(num.tag_name)

span
span
span
span
span
span
span
span
span
span
span
span
span
span
span
span
span
span
span
span
span
span
span
span
span
span
span
span
span
span
span
a
a
a
a
a
a
a
a
a
a


In [7]:
for num in _nums:
    if num.tag_name == 'a':
        print(num.get_attribute('href'))

javascript:void(0);
http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&query=%EC%BD%94%EB%A1%9C%EB%82%98&queryText=&iStartCount=0&iGroupView=5&icate=all&colName=bib_t&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&pageScale=10&orderBy=&fsearchMethod=search&isFDetailSearch=N&sflag=1&searchQuery=%EC%BD%94%EB%A1%9C%EB%82%98&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&resultKeyword=%EC%BD%94%EB%A1%9C%EB%82%98&pageNumber=1&p_year1=&p_year2=&dorg_storage=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&language_code=&ccl_code=&language=&inside_outside=&fric_yn=&image_yn=&regnm=&gubun=&kdc=&ttsUseYn=#redirect
http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&query=%EC%BD%94%EB%A1%9C%EB%82%98&queryText=&iStartCount=0&iGroupView=5&icate=all&colName=bib_t&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&pageScale=10&orderBy=&fsearchMethod=search

<img src='./img1.png'><br>
```class``` 이름이 ```num```인 것들 중 ```a``` tag인 것들만이 또 다른 페이지로 이동이 가능한  ```Elements```이다.<br>
더 나아가, 위 사진처럼 첫번째 태그는 현재 내가 위치한 페이지이므로 별도 url이 아니라 ```javascript:void(0);```이라 하여 페이지 이동이 없게 처리되어 있다. <br>이러한 것을 고려하여 현재 보이는 모든 페이지의 url을 긁어서 더 많이 크롤링하도록 해보자.

In [46]:
nums = driver.find_elements_by_class_name('num')
nums = [num.get_attribute('href') for num in nums if num.tag_name == 'a']
print('페이지 수 :', len(nums))
print('페이지 내용 :')
for num in nums:
    print(num)

페이지 수 : 10
페이지 내용 :
javascript:void(0);
http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&query=%EC%BD%94%EB%A1%9C%EB%82%98&queryText=&iStartCount=0&iGroupView=5&icate=all&colName=bib_t&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&pageScale=10&orderBy=&fsearchMethod=search&isFDetailSearch=N&sflag=1&searchQuery=%EC%BD%94%EB%A1%9C%EB%82%98&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&resultKeyword=%EC%BD%94%EB%A1%9C%EB%82%98&pageNumber=1&p_year1=&p_year2=&dorg_storage=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&language_code=&ccl_code=&language=&inside_outside=&fric_yn=&image_yn=&regnm=&gubun=&kdc=&ttsUseYn=#redirect
http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&query=%EC%BD%94%EB%A1%9C%EB%82%98&queryText=&iStartCount=0&iGroupView=5&icate=all&colName=bib_t&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&pageScale=10&orderBy=&

In [48]:
pd.Series(nums).unique()

array(['javascript:void(0);',
       'http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&query=%EC%BD%94%EB%A1%9C%EB%82%98&queryText=&iStartCount=0&iGroupView=5&icate=all&colName=bib_t&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&pageScale=10&orderBy=&fsearchMethod=search&isFDetailSearch=N&sflag=1&searchQuery=%EC%BD%94%EB%A1%9C%EB%82%98&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&resultKeyword=%EC%BD%94%EB%A1%9C%EB%82%98&pageNumber=1&p_year1=&p_year2=&dorg_storage=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&language_code=&ccl_code=&language=&inside_outside=&fric_yn=&image_yn=&regnm=&gubun=&kdc=&ttsUseYn=#redirect'],
      dtype=object)

In [50]:
urls = []
def get_urls_by_multi_pages():
    nums = driver.find_elements_by_class_name('num')
    nums = [num.get_attribute('href') for num in nums if num.tag_name == 'a']
    nums = nums[1:]
    # 첫번째 페이지는 javascript:void(0);이므로 별도 처리 필요.
    for i in range(1, 11):
        page = '/html/body/div[1]/div[2]/div[4]/div[2]/div/div[3]/div[2]/ul/li['+str(i)+']/div[2]/p[1]/a'
        url=driver.find_element_by_xpath(page).get_attribute('href')
        urls.append(url)
    # 두번째 페이지부터 마지막 페이지(max 10)까지 순차적으로 이동하여 url 크롤링
    for num in nums:
        driver.get(num)
        sleep(1)
        for i in range(1, 11):
            page = '/html/body/div[1]/div[2]/div[4]/div[2]/div/div[3]/div[2]/ul/li['+str(i)+']/div[2]/p[1]/a'
            url=driver.find_element_by_xpath(page).get_attribute('href')
            urls.append(url)

In [51]:
get_urls_by_multi_pages()

In [52]:
len( pd.Series(urls).unique() )

10

페이지로 이동하는 ```num```이라는 ```anchor``` tag의  받아서 각각 이동하여 논문의 url을 크롤링하려 했지만,<br>
위에서 확인할 수 있듯이, 모든 페이지가 동일한 ```href``` 값을 가진다. <br>
이는 아마도 ```javascript```로 처리하였기때문인 것 같다. 이를 타파하기 위해서는 <br>
```onclick``` 속성을 이용해야 할 것 같다.

In [53]:
//*[@id="divContent"]/div[2]/div/div[4]/a[2]  # min: 2 -> 현재 페이지
//*[@id="divContent"]/div[2]/div/div[4]/a[11] # max: 11
# 페이지가 하나일 경우 현재 페이지만 크롤링
# 그 외에는 #page - 1 만큼 다음 페이지로 이동하여 크롤링

['javascript:void(0);',
 'http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&query=%EC%BD%94%EB%A1%9C%EB%82%98&queryText=&iStartCount=0&iGroupView=5&icate=all&colName=bib_t&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&pageScale=10&orderBy=&fsearchMethod=search&isFDetailSearch=N&sflag=1&searchQuery=%EC%BD%94%EB%A1%9C%EB%82%98&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&resultKeyword=%EC%BD%94%EB%A1%9C%EB%82%98&pageNumber=1&p_year1=&p_year2=&dorg_storage=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&language_code=&ccl_code=&language=&inside_outside=&fric_yn=&image_yn=&regnm=&gubun=&kdc=&ttsUseYn=#redirect',
 'http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&query=%EC%BD%94%EB%A1%9C%EB%82%98&queryText=&iStartCount=0&iGroupView=5&icate=all&colName=bib_t&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&pageScale=10&orderBy=&fsearchMet

In [69]:
urls = []
def get_urls_by_multi_pages():
    nums = driver.find_elements_by_class_name('num')
    nums = [num.get_attribute('href') for num in nums if num.tag_name == 'a']
    # 전체 페이지 수를 num_pages에 저장. maximum 10, minimum 1
    num_pages = len(nums) 
    
    # 전체 페이지가 1개이면 for문은 작동x, 전체 페이지가 2 이상 10이하이면 모든 페이지에 대해 크롤링.
    for i in range(num_pages):
        # 다음 페이지로 이동, 첫번째 페이지에 한해서 아무런 반응 없음(javascript:void(0);)
        driver.find_element_by_xpath(f'//*[@id="divContent"]/div[2]/div/div[4]/a[{2+i}]').send_keys(Keys.ENTER)
        # 해당 페이지에서 크롤링
        _get_url_in_current_page()
    
def _get_url_in_current_page():
    # 해당 페이지에 표시되는 논문 수를 total에 저장. xpath에 li의 인덱스를 표시하지 않으면 같은 레벨의 Elements수를 반환한다.
    total = len(driver.find_elements_by_xpath('//*[@id="divContent"]/div[2]/div/div[3]/div[2]/ul/li'))
    if total < 10:
        # 주어진 키워드로 검색 시 전체 논문 수가 10개 미만인 경우 - 논문 첫 페이지의 논문 수 = 전체 논문 수
        for i in range(1, total+1):  
            page = '/html/body/div[1]/div[2]/div[4]/div[2]/div/div[3]/div[2]/ul/li['+str(i)+']/div[2]/p[1]/a'
            url=driver.find_element_by_xpath(page).get_attribute('href')
            urls.append(url)
    else:
        # 주어진 키워드로 검색 시 전체 논문 수가 10개 이상인 경우 - 논문 첫 페이지의 논문 수 = 10개
        for i in range(1, 11):
            page = '/html/body/div[1]/div[2]/div[4]/div[2]/div/div[3]/div[2]/ul/li['+str(i)+']/div[2]/p[1]/a'
            url=driver.find_element_by_xpath(page).get_attribute('href')
            urls.append(url)

In [70]:
get_urls_by_multi_pages()

```get_urls_by_multi_pages()```는 RISS에서 검색했을 때, double arrow가 나오기 전까지 크롤링 하도록 설계되었다.
즉 최소 0개부터 최대 100개까지의 논문을 크롤링할 수 있다.
만약 더 많은 논문을 원한다면 ```get_urls_by_multi_pages()```를 한 번 실행하고 next double arrow를 찾아서
Keys.ENTER를 누르도록 한 후 다시 ```get_urls_by_multi_pages()```를 반복하도록 설계하면 될 것이다.

In [72]:
len(pd.Series(urls).unique())

100

In [81]:
def store(url):
    data=[]  #빈 리스트
    authur=""  #저자
    year=""  #연도
    title=""  #제목
    first_info=""  #부가정보
        
    #저자
    author = driver.find_element_by_xpath('//*[@id="thesisInfoDiv"]/div[2]/div[1]/ul/li[1]/div/p/a').text
        
    #연도
    year = driver.find_element_by_xpath('//*[@id="thesisInfoDiv"]/div[2]/div[1]/ul/li[4]/div/p').text
        
    #제목
    title = driver.find_element_by_xpath('//*[@id="thesisInfoDiv"]/div[1]/h3').text
             
    #부가정보    
    moreViews = driver.find_elements_by_class_name('moreView')

    for moreView in moreViews:
        moreView.send_keys(Keys.ENTER)
        sleep(0.1)

    try:
        first_info = driver.find_element_by_xpath('//*[@id="additionalInfoDiv"]/div/div[1]/div[3]/p').text
    except:
        # 부가정보가 없을 때, data.append를 할 게 아니라, first_info에 값을 저장해야 한다.
        first_info = "부가정보가 없습니다."
        
    data.append(author)
    data.append(int(year))
    data.append(title)
    data.append(first_info)
    data.append(url)
    paper.append(data)

In [82]:
paper=[]
for url in urls:
    driver.get(url)
    store(url)
    sleep(1)

driver.close()
    
paper_info = pd.DataFrame(paper, columns=['저자', '발행년도', '제목', '부가정보', 'Url'])

In [85]:
paper_info.to_csv('./paper_info.csv', encoding="utf-8-sig") #  encoding='utf-8'이라고 하면 한글이 깨지더라.

# Check!

In [9]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(pd.read_csv('./paper_info.csv',index_col=0))

,저자,발행년도,제목,부가정보,Url
0,이상순,2021,코로나바이러스감염증-19와 보육교사의 심리적 스트레스에 관한 연구 = A Study...,코로나바이러스감염증-19가 확산되면서 우리나라를 비롯한 전 세계는 펜데믹(Pande...,http://www.riss.kr/search/detail/DetailView.do...
1,이승윤,2021,코로나-19로 인한 호텔조리종사원의 근무스케줄 변화가 조직몰입에 미치는 영향 : 스...,현재 우리나라뿐만 아니라 전 세계가 코로나-19로 인한 감영병 사태로 인해 경제부문...,http://www.riss.kr/search/detail/DetailView.do...
2,김경주,2020,국가 재난의 위기 속에서 한국교회의 대응 방안에 관한 연구 : 소망교회의 코로나19...,부가정보가 없습니다.,http://www.riss.kr/search/detail/DetailView.do...
3,손희나,2020,코로나19(COVID-19)로 인한 어린이집원장의 어려움과 코로나19에 대한 대응방...,The purpose of this study was to find out the ...,http://www.riss.kr/search/detail/DetailView.do...
4,서신,2021,코로나19 대응정책에서 나타난 차별의 연구 : 이주노동자를 중심으로,이 연구는 한국정부의 이주노동자에 대한 방역대응과 재난지원 정책에 있어 차별적인 요...,http://www.riss.kr/search/detail/DetailView.do...
5,배은정,2021,코로나19 대유행에 따른 유아의 일상생활 변화와 어머니의 코로나19에 대한 걱정이 ...,The present study examined the effects the dai...,http://www.riss.kr/search/detail/DetailView.do...
6,박사랑,2021,코로나바이러스감염증-19(COVID-19)가 공연예술에 미친 영향과 대응 사례에 관...,본 연구는 코로나바이러스감염증-19 발생에 따른 공연예술 분야 피해와 변화를 파악하...,http://www.riss.kr/search/detail/DetailView.do...
7,Liu Jia-Qi,2021,코로나19 시대의 한국관광에 대한 중국인의 행태 고찰 = A Study on the...,As the risk of face-to-face contact with Covid...,http://www.riss.kr/search/detail/DetailView.do...
8,김희숙,2021,코로나 -19 대유행으로 인한 노인 정신건강과 신체활동 실태조사 = The Stud...,본 연구의 목적은 지역사회에 거주하는 65세 이상 노인의 정신건강에 코로나-19 대...,http://www.riss.kr/search/detail/DetailView.do...
9,박지원,2021,코로나19 상황에서의 미디어 이용과 미디어 신뢰가 코로나19 감염 가능성에 대한 낙...,부가정보가 없습니다.,http://www.riss.kr/search/detail/DetailView.do...
